In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_17_9_0,0.995269,0.990418,0.986482,0.989590,0.004731,0.018988,0.011476,0.015453,"Hidden Size=[45], regularizer=0.01, learning_r..."
1,model_17_9_1,0.995251,0.990345,0.986457,0.989531,0.004749,0.019132,0.011498,0.015540,"Hidden Size=[45], regularizer=0.01, learning_r..."
2,model_17_9_2,0.995233,0.990279,0.986433,0.989479,0.004767,0.019262,0.011518,0.015618,"Hidden Size=[45], regularizer=0.01, learning_r..."
3,model_17_9_3,0.995216,0.990220,0.986411,0.989431,0.004784,0.019380,0.011536,0.015689,"Hidden Size=[45], regularizer=0.01, learning_r..."
4,model_17_9_4,0.995201,0.990166,0.986391,0.989387,0.004799,0.019487,0.011553,0.015753,"Hidden Size=[45], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
414,model_13_7_3,0.968358,0.960257,0.965751,0.966577,0.031642,0.033552,0.043201,0.038093,"Hidden Size=[35], regularizer=0.01, learning_r..."
453,model_15_9_4,0.966107,0.942623,0.931093,0.942786,0.033893,0.100381,0.012850,0.059190,"Hidden Size=[40], regularizer=0.01, learning_r..."
482,model_13_7_2,0.964709,0.963853,0.969013,0.969686,0.035291,0.030516,0.039085,0.034549,"Hidden Size=[35], regularizer=0.01, learning_r..."
484,model_5_9_0,0.964441,0.991674,0.978105,0.985446,0.035559,0.008088,0.022519,0.014879,"Hidden Size=[15], regularizer=0.01, learning_r..."
